In [ ]:
# display prompt (only configurable in terminal)

see_this = 'https://github.com/ipython/ipython/wiki/Cookbook:-Dynamic-prompt' # printed to gdrive..

In [32]:
# update 'all_notebooks.html'
from os import listdir,getcwd
from os.path import isfile, join

cwd = getcwd()
notebooks = [ f for f in listdir(cwd) if (isfile(join(cwd,f)) and f.find('.ipynb')!=-1 and f[0]!='.') ]

!echo "" > all_notebooks.html
with open('all_notebooks.html', 'a') as f:
    f.write('<html>')
    for it in notebooks:
        a=!/Users/admin/SERVER2/BD_Scripts/ipython/ENV/bin/ipython nbconvert --profile nbserver --to html --template basic $it --stdout
        f.write('\n'.join(a[4:]))
    f.write('</html>')
# %less all_notebooks.html

In [1]:
# ipy-history
import pandas as pd
pd.set_option('expand_frame_repr',False)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.width',180)
np = pd.np
np.set_printoptions(linewidth=200,threshold=np.nan)
import sqlite3
c = get_ipython()
h_path = str(c.config['ProfileDir']['location'])+'/history.sqlite'
con = sqlite3.connect(h_path)
df = pd.read_sql("""
    select h.*,s.start,s.end from history h
    inner join sessions s on h.session=s.session
                 """, con)
df['first_lines'] = df.source_raw.map(lambda s: s[:s.find('\n')])
# df['focus'] = df.first_lines.map(lambda s: True if s.lower().find('divide')!=-1 else False)
# df[df.focus==True].sort(['session','line'],ascending=[True,True])

In [2]:
df.sort(['start','end'],ascending=[False,False])

# z= df[df.session==413].ix[:,'source'].tolist()
# s=''
# for it in z:
#     s+=it
# print s

,session,line,source,source_raw,start,end,first_lines
17338,804,1,# ipy-history\nimport pandas as pd\npd.set_opt...,# ipy-history\nimport pandas as pd\npd.set_opt...,2015-02-08 03:33:17.771105,None,# ipy-history
17314,799,1,## Open Blocking Connection:\nimport pika\n\n\...,## Open Blocking Connection:\nimport pika\n\n\...,2015-02-05 17:08:14.329292,None,## Open Blocking Connection:
17315,799,2,get_ipython().magic(u'pip_install pika'),%pip_install pika,2015-02-05 17:08:14.329292,None,%pip_install pik
17316,799,3,## Open Blocking Connection:\nimport pika\n\n\...,## Open Blocking Connection:\nimport pika\n\n\...,2015-02-05 17:08:14.329292,None,## Open Blocking Connection:
17317,799,4,ch = open_connection(),ch = open_connection(),2015-02-05 17:08:14.329292,None,ch = open_connection(
17318,799,5,ch.basic_publish(exchange='celery@worker.git_s...,ch.basic_publish(exchange='celery@worker.git_s...,2015-02-05 17:08:14.329292,None,ch.basic_publish(exchange='celery@worker.git_s...
17319,799,6,"ch.basic_publish(exchange='celery',\n ...","ch.basic_publish(exchange='celery',\n ...",2015-02-05 17:08:14.329292,None,"ch.basic_publish(exchange='celery',"
17320,799,7,"ch.basic_publish(exchange='celery',\n ...","ch.basic_publish(exchange='celery',\n ...",2015-02-05 17:08:14.329292,None,"ch.basic_publish(exchange='celery',"
17321,799,8,ch.close(),ch.close(),2015-02-05 17:08:14.329292,None,ch.close(
17322,799,9,ch = open_connection(),ch = open_connection(),2015-02-05 17:08:14.329292,None,ch = open_connection(


In [ ]:
# pt,stop=9446,False
# while stop==False:
#     t = df.ix[pt,'source']
#     if t.find("s.split('-')[0],a")!=-1:
#         print pt
#         print t
#         stop=True
#         break
#     else:
#         pt-=1
df.ix[9300:9400,'source']

In [38]:
# qtconsole/ssh
def ib_k(a='get_ipython()'):
    from os import system as os_cmd
    from os import path as os_path
    from subprocess import Popen as sub_popen
    from subprocess import PIPE as sub_PIPE

    # 1. get info about kernel
    a=get_ipython()
    fpath = a.kernel.config['IPKernelApp']['connection_file']
    fpath_shared = fpath[fpath.find('.ipython'):]
    fpath_in_mnt = '/Volumes/'+fpath_shared
    proc = sub_popen(["cat "+fpath], stdout=sub_PIPE, shell=True)
    (t, err) = proc.communicate()
    k_info = eval(t)

    # 2. confirm access to REMOTE
    if not os_path.isdir('/Volumes/SERVER3'):
        cmd  = 'mkdir -p /Volumes/SERVER3; '
        cmd += '/opt/local/bin/sshfs mbp2:/Users/admin/SERVER3 /Volumes/SERVER3 -ovolname=SERVER3;'
        os_cmd(cmd)

    # 3. confirm access from REMOTE
    if not os_path.isdir('/Volumes/mbp2'):
        cmd  = 'mkdir -p /Volumes/mbp2; '
        cmd += '/opt/local/bin/sshfs mbp2:/ /Volumes/mbp2 -ovolname=mbp2;'
        os_cmd(cmd)
    if not os_path.isdir('/Volumes/mbp2/Volumes/.ipython'):
        cmd  = "ssh mbp2 'mkdir -p /Volumes/.ipython'"
        os_cmd(cmd)
        cmd = "ssh mbp2 '/opt/local/bin/sshfs mb:/Users/admin/.ipython /Volumes/.ipython -ovolname=.ipython;'"        
        os_cmd(cmd)
    
    # 4. create script on REMOTE for execution

    # ---- add cmds for opening proper ports
    cmds = []
    for k,v in k_info.iteritems():
        if k.find('_port')!=-1:
            cmds.append('ssh mb -f -N -L '+str(v)+':localhost:'+str(v)+';')

    # ---- add cmd for opening proper connection
    cmds.append("/Users/admin/SERVER2/BD_Scripts/ipython/ENV/bin/ipython qtconsole --existing %s"%fpath_in_mnt+";")

    # ---- add cmds for closing connection
    for k,v in k_info.iteritems():
        if k.find('_port')!=-1:
            cmds.append("kill `ps -A | grep "+str(v)+":localhost:"+str(v)+" | grep -v grep | awk '{print $1}'`"+'; ')
    
    # ---- push cmds to REMOTE
    f = open('/Volumes/SERVER3/scripts/k.sh','w')
    for it in cmds: f.write(it+'\n')
    f.close()

    # 5. send ssh cmd for REMOTE execution of script
    os_cmd("ssh mbp2 '~/SERVER3/scripts/k.sh' &")

ib_k(a='get_ipython()')

In [ ]:
# file links
from IPython.display import FileLink, FileLinks
a=FileLinks('.')
a

In [ ]:
# get_bash_aliases
def get_bash_aliases():

    import re
    import os.path
    d = []
    with open(os.path.expanduser('~/.alias')) as bashrc:
        for line in bashrc:
            if not line.startswith('alias'):
                continue

            parts = re.match(r'^alias (\w+)=([\'"]?)(.+)\2$', line.strip())
            if not parts:
                continue

            source, _, target = parts.groups()
            if target.find('cd')==0: target=target[3:]
            d.append({'alias':source,
                      'cmd':target})
#             print source,target
#             chk = raw_input('Save \t"'+source+'"\tas\t"'+target+'"')
#             if chk=='y':
#                 %alias source `$target`
#             elif chk=='q':
#                 break
    import pandas as pd
    return pd.DataFrame(d).sort('alias')

them = get_bash_aliases()

In [9]:
for it in zip(them.alias.tolist(),them.cmd.tolist()): print it[0],'\t\t',it[1]

AP_android 		/Users/admin/django/Dropbox/aporo/static/sencha/phonegap/platforms/android/
AP_phonegap_build_android 		AP_android; sudo phonegap build android --release;
AP_sencha 		/Users/admin/django/Dropbox/aporo/static/sencha/
AP_sencha_build 		AP_sencha; sudo sencha app build package;
CYB 		CYB
JSON 		f_getJSON
RD 		RD
RDt 		RDt
apache_compile 		apachectl -V
apache_config 		sudo emacs /etc/apache2/httpd.conf
apache_load 		sudo launchctl load -w /System/Library/LaunchDaemons/org.apache.httpd.plist
apache_mods 		apachectl -M
apache_restart 		sudo apachectl restart
apache_test 		sudo apachectl configtest
apache_unload 		sudo launchctl unload -w /System/Library/LaunchDaemons/org.apache.httpd.plist
apache_virtuals 		apachectl -S
aporo 		sv_load; ng_load;
bashedit 		edit_source_bash
bashedit_prof 		sudo emacs ~/.bash_profile; source ~/.bash_profile;
bashedit_source 		source ~/.bash_profile
btoff 		echo "admin" | sudo defaults write /Library/Preferences/com.apple.Bluetooth.plist Controller

In [ ]:
a = %alias
import pandas as pd
j = pd.DataFrame(a,columns=['alias','cmd']).sort('alias')
j.ix[5,'cmd']

In [13]:
### Install &|& Link Python Libraries
a=0
this_mod = 'numexpr'

# %open_mod $this_mod
# %link_lib_to_ipy $this_mod 
# !printf "ln -s ";printf `/opt/local/bin/python -c "import imp; print imp.find_module('numexpr')[1]" `;printf " ";printf `which python` | printf `sed 's|/bin/python||g'` ; printf "/lib/python2.7/site-packages"
# !which ipython
# a = /
# !$(printf "ln -s "; printf `/opt/local/bin/python -c "import imp; print imp.find_module('numexpr')[1]" `;printf " ";printf `which python` | printf `sed 's|/bin/python||g'` ; printf "/lib/python2.7/site-packages")
# a = !pip search $this_mod
# !pip install $this_mod

# !sudo port search $this_mod
# this_port = 'py27-shapely@1.3.3'
# !port variant $this_port
# !sudo port install $this_port
# a



In [ ]:
### Aliases & Maintenance

# %rehashx?

# %alias open_mod `printf "open " ; /opt/local/bin/python -c "import imp; print imp.find_module('%s')[1]"`
# %store open_mod

# %alias link_lib_to_ipy $(printf "ln -s "; printf `/opt/local/bin/python -c "import imp; print imp.find_module('%s')[1]" `;printf " ";printf `which python` | printf `sed 's|/bin/python||g'` ; printf "/lib/python2.7/site-packages")
# %store link_lib_to_ipy

# %alias download_untar $(echo "`python -c \"x='%s'; print x,x[x.rfind('/')+1:]\"`" \
#            | awk '{ print "/opt/local/bin/wget "$1" ; tar -zxvf "$2"; rm "$2 }' \
#            | sh)
# %store download_untar

####General

    %run
    %hist 
    %quickref
    %matplotlib inline
    %qtconsole
    %debug
    %history
    *int*?
    %%writefile test.txt
    !pwd

####Debug

    import pdb; pdb.set_trace()

####Modules

Loading:

    %load_ext autoreload
    %autoreload 2
    
    %load http://pastebin.com/raw.php?i=mGiV1FwY
    
        - %pastebin "cell one" 0-10


    my_module = reload(my_module)

Call From Module:

    from IPython import embed; embed()

Show Links for Local Files:

    from IPython.display import FileLink, FileLinks
    FileLinks('.', notebook_display_formatter=True)
    FileLinks('output/')

QT-Console:

    ipython qtconsole --existing
    %connect_info


Pprint Example:

    from pprint import pprint

    def print_variables_of(obj):
        """variables of an object"""
        pprint (vars(obj))

    def print_callables_of(obj):
        """callables (functions too) of an object"""
        li = []
        for name in dir(obj):
            attr = getattr(obj, name)
            if hasattr(attr, '__call__'):
                li.append(name)
        pprint(li)

IPython -- An enhanced Interactive Python - Quick Reference Card
---

obj?, objjQuery20308880275620613247_1416088857606      : Get help, or more help for object (also works as ?obj, jQuery20306815248038619757_1416089336937obj).
?foo.*abc*       : List names in 'foo' containing 'abc' in them.
%magic           : Information about IPython's 'magic' % functions.

Magic functions are prefixed by % or %%, and typically take their arguments
without parentheses, quotes or even commas for convenience.  Line magics take a
single % and cell magics are prefixed with two %%.

Example magic function calls:

%alias d ls -F   : 'd' is now an alias for 'ls -F'
alias d ls -F    : Works if 'alias' not a python name
alist = %alias   : Get list of aliases to 'alist'
cd /usr/share    : Obvious. cd -<tab> to choose from visited dirs.
%cdjQuery203016718862811103463_1416089461856            : See help AND source for magic %cd
%timeit x=10     : time the 'x=10' statement with high precision.
%%timeit x=2**100
x**100           : time 'x*100' with a setup of 'x=2**100'; setup code is not
                   counted.  This is an example of a cell magic.

System commands:

!cp a.txt b/     : System command escape, calls os.system()
cp a.txt b/      : after %rehashx, most system commands work without !
cp ${f}.txt $bar : Variable expansion in magics and system commands
files = !ls /usr : Capture sytem command output
files.s, files.l, files.n: "a b c", ['a','b','c'], 'a\nb\nc'

History:

_i, _ii, _iii    : Previous, next previous, next next previous input
_i4, _ih[2:5]    : Input history line 4, lines 2-4
exec _i81        : Execute input history line #81 again
%rep 81          : Edit input history line #81
_, __, ___       : previous, next previous, next next previous output
_dh              : Directory history
_oh              : Output history
%hist            : Command history. '%hist -g foo' search history for 'foo'

Autocall:

f 1,2            : f(1,2)  # Off by default, enable with %autocall magic.
/f 1,2           : f(1,2) (forced autoparen)
,f 1 2           : f("1","2")
;f 1 2           : f("1 2")

Remember: TAB completion works in many contexts, not just file names
or python names.

The following magic functions are currently available:

%alias:
    Define an alias for a system command.
%alias_magic:
    %alias_magic [-l] [-c] name target
%autocall:
    Make functions callable without having to type parentheses.
%automagic:
    Make magic functions callable without having to type the initial %.
%autosave:
    Set the autosave interval in the notebook (in seconds).
%bookmark:
    Manage IPython's bookmark system.
%cd:
    Change the current working directory.
%clear:
    Clear the terminal.
%cls:
    Clear the terminal.
%colors:
    Switch color scheme for prompts, info system and exception handlers.
%config:
    configure IPython
%connect_info:
    Print information for connecting other clients to this kernel
%debug:
    %debug [--breakpoint FILE:LINE] [statement [statement ...]]
%dhist:
    Print your history of visited directories.
%dirs:
    Return the current directory stack.
%doctest_mode:
    Toggle doctest mode on and off.
%ed:
    Alias for `%edit`.
%edit:
    Bring up an editor and execute the resulting code.
%env:
    List environment variables.
%gui:
    Enable or disable IPython GUI event loop integration.
%guiref:
    Show a basic reference about the GUI Console.
%hist:
    Alias for `%history`.
%history:
    %history [-n] [-o] [-p] [-t] [-f FILENAME] [-g [PATTERN [PATTERN ...]]]
%install_default_config:
    %install_default_config has been deprecated.
%install_ext:
    Download and install an extension from a URL, e.g.::
%install_profiles:
    %install_profiles has been deprecated.
%killbgscripts:
    Kill all BG processes started by %%script and its family.
%less:
    Show a file through the pager.
%load:
    Load code into the current frontend.
%load_ext:
    Load an IPython extension by its module name.
%loadpy:
    Alias of `%load`
%logoff:
    Temporarily stop logging.
%logon:
    Restart logging.
%logstart:
    Start logging anywhere in a session.
%logstate:
    Print the status of the logging system.
%logstop:
    Fully stop logging and close log file.
%lsmagic:
    List currently available magic functions.
%macro:
    Define a macro for future re-execution. It accepts ranges of history,
%magic:
    Print information about the magic function system.
%matplotlib:
    %matplotlib [gui]
%more:
    Show a file through the pager.
%notebook:
    %notebook [-e] [-f FORMAT] filename
%page:
    Pretty print the object and display it through a pager.
%pastebin:
    Upload code to Github's Gist paste bin, returning the URL.
%pdb:
    Control the automatic calling of the pdb interactive debugger.
%pdef:
    Print the call signature for any callable object.
%pdoc:
    Print the docstring for an object.
%pfile:
    Print (or run through pager) the file where an object is defined.
%pinfo:
    Provide detailed information about an object.
%pinfo2:
    Provide extra detailed information about an object.
%popd:
    Change to directory popped off the top of the stack.
%pprint:
    Toggle pretty printing on/off.
%precision:
    Set floating point precision for pretty printing.
%profile:
    Print your currently active IPython profile.
%prun:
    Run a statement through the python code profiler.
%psearch:
    Search for object in namespaces by wildcard.
%psource:
    Print (or run through pager) the source code for an object.
%pushd:
    Place the current dir on stack and change directory.
%pwd:
    Return the current working directory path.
%pycat:
    Show a syntax-highlighted file through a pager.
%pylab:
    %pylab [--no-import-all] [gui]
%qtconsole:
    Open a qtconsole connected to this kernel.
%quickref:
    Show a quick reference sheet 
%recall:
    Repeat a command, or get command to input line for editing.
%rehashx:
    Update the alias table with all executable files in $PATH.
%reload_ext:
    Reload an IPython extension by its module name.
%rep:
    Alias for `%recall`.
%rerun:
    Re-run previous input
%reset:
    Resets the namespace by removing all names defined by the user, if
%reset_selective:
    Resets the namespace by removing names defined by the user.
%run:
    Run the named file inside IPython as a program.
%save:
    Save a set of lines or a macro to a given filename.
%sc:
    Shell capture - run shell command and capture output (DEPRECATED use !).
%store:
    Lightweight persistence for python variables.
%sx:
    Shell execute - run shell command and capture output (!! is short-hand).
%system:
    Shell execute - run shell command and capture output (!! is short-hand).
%tb:
    Print the last traceback with the currently active exception mode.
%time:
    Time execution of a Python statement or expression.
%timeit:
    Time execution of a Python statement or expression
%unalias:
    Remove an alias
%unload_ext:
    Unload an IPython extension by its module name.
%who:
    Print all interactive variables, with some minimal formatting.
%who_ls:
    Return a sorted list of all interactive variables.
%whos:
    Like %who, but gives some extra information about each variable.
%xdel:
    Delete a variable, trying to clear it from anywhere that
%xmode:
    Switch modes for the exception handlers.
%%!:
    Shell execute - run shell command and capture output (!! is short-hand).
%%HTML:
    Alias for `%%html`.
%%SVG:
    Alias for `%%svg`.
%%bash:
    %%bash script magic
%%capture:
    %capture [--no-stderr] [--no-stdout] [output]
%%cmd:
    %%cmd script magic
%%debug:
    %debug [--breakpoint FILE:LINE] [statement [statement ...]]
%%file:
    Alias for `%%writefile`.
%%html:
    Render the cell as a block of HTML
%%javascript:
    Run the cell block of Javascript code
%%latex:
    Render the cell as a block of latex
%%perl:
    %%perl script magic
%%powershell:
    %%powershell script magic
%%prun:
    Run a statement through the python code profiler.
%%pypy:
    %%pypy script magic
%%python:
    %%python script magic
%%python3:
    %%python3 script magic
%%ruby:
    %%ruby script magic
%%script:
    %shebang [--proc PROC] [--bg] [--err ERR] [--out OUT]
%%sh:
    %%sh script magic
%%svg:
    Render the cell as an SVG literal
%%sx:
    Shell execute - run shell command and capture output (!! is short-hand).
%%system:
    Shell execute - run shell command and capture output (!! is short-hand).
%%time:
    Time execution of a Python statement or expression.
%%timeit:
    Time execution of a Python statement or expression
%%writefile:
    %writefile [-a] filename
